In [1]:
import os 
from pathlib import Path
import sys
import subprocess
import time
from typing import Optional
from collections import defaultdict 
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import json
from IPython.display import display
import latextable
from texttable import Texttable
from arguments_dict import arguments_dict

executables = ['genmc', 'genmc-wkmo', 'genmc-xmm']
executable_paths_dict = dict([(exe, Path("executables") / exe) for exe in executables])
test_names = ['LBn+ctrl', 'LBn+data', 'LBn', 'LBn-pairs']
tests = dict([(t, Path('tests') / t / 'variants' / (os.listdir(Path('tests') / t / 'variants')[0])) for t in test_names if os.path.isdir(Path('tests') / t)])
subprocess_timeout = 1000000 #s

class RunExecutableResult:
    class Timeout:
        pass
    class Ok:
        def __init__(self, output) -> None:
            super().__init__()
            self.output = output
    class Error:
        def __init__(self, retcode, output) -> None:
            super().__init__()
            self.retcode = retcode
            self.output = output

def run_executable(exe_path: Path, args: list[str], test_path: Path) -> RunExecutableResult:
    try:
        result = subprocess.run([exe_path, *args, test_path], stdout=subprocess.PIPE, stderr=subprocess.PIPE, timeout=subprocess_timeout)
        result_stdout = result.stdout.decode()
        result_stderr = result.stderr.decode()
        if result.returncode == 0:
            return RunExecutableResult.Ok(result_stdout + result_stderr)
        else:
            return RunExecutableResult.Error(result.returncode, result_stdout + result_stderr)
    except subprocess.TimeoutExpired:
        return RunExecutableResult.Timeout()

In [2]:
def get_duplicate_executions(string: str) -> Optional[int]:
    try:
        pattern = "Number of duplicate executions explored: "
        i = string.index(pattern) + len(pattern)
        number_str = ""
        while i < len(string) and string[i].isdigit():
            number_str += string[i]
            i += 1
        return int(number_str)
    except:
        return None

assert(get_duplicate_executions("abc 123 Number of duplicate executions explored: 123456") == 123456)

def get_number_executions(string: str) -> Optional[int]:
    pattern = "Number of complete executions explored: "
    i = string.index(pattern) + len(pattern)
    number_str = ""
    while i < len(string) and string[i].isdigit():
        number_str += string[i]
        i += 1
    try:
        return int(number_str)
    except:
        return None

assert(get_number_executions("abc 123 Number of complete executions explored: 123456") == 123456)

def get_number_lb_races(string: str) -> Optional[int]:
    pattern = "Number of load buffering races explored: "
    try:
        i = string.index(pattern) + len(pattern)
        number_str = ""
        while i < len(string) and string[i].isdigit():
            number_str += string[i]
            i += 1
        return int(number_str)
    except:
        return None

class ResultItem:
    def __init__(self, execs, dups, lb_races) -> None:
        self.execs = execs
        self.dups = dups
        self.lb_races = lb_races

def run_and_get_results(exe_path: Path, args: list[str], test_path: Path) -> RunExecutableResult:
    res = run_executable(exe_path, args, test_path)
    if isinstance(res, RunExecutableResult.Ok):
        execs = get_number_executions(res.output)
        dups = get_duplicate_executions(res.output)
        lb_races = get_number_lb_races(res.output)
        if execs is None:
            # print(f"could not get number of executions in {res.output}")
            execs = 0
        if dups is None:
            # print(f"could not get duplicate executions in {res.output}")
            dups = 0
        return RunExecutableResult.Ok(ResultItem(execs, dups, lb_races))
    else:
        return res

In [3]:
timeout = '{\\fontspec{Symbola}\\symbol{"231B}}'
rows = [["Test Name", "GenMC Execs", "WMC Execs", "WMC Dupls", "XMM Execs", "XMM Dupls", "LB Races"]]
assert(executables == ['genmc', 'genmc-wkmo', 'genmc-xmm'])
for test_name in test_names:
    for n in [10, 12, 14]:
        row = [f"{test_name}({n})"]
        for exe in executables:
            args = arguments_dict[exe] + ["--", f"-D N={n}"]
            res = run_and_get_results(executable_paths_dict[exe], args, tests[test_name])
            if isinstance(res, RunExecutableResult.Ok):
                execs = res.output.execs
                dups = res.output.dups
                lb_races = res.output.lb_races
                row += [execs, dups] if exe != "genmc" else [execs]
                if lb_races is not None:
                    row += [lb_races]
            elif isinstance(res, RunExecutableResult.Timeout):
                row += [timeout] * (2 if exe != "genmc" else 1)
            elif isinstance(res, RunExecutableResult.Error):
                print("Error: {res.output}")
                exit(1)
            else:
                assert(0)
        rows.append(row)

In [4]:
table_1 = Texttable()
table_1.set_cols_align(["l"] + ["c" for i in range(len(rows[0]) - 1)])
table_1.set_cols_valign(["m" for i in range(len(rows[0]))])
table_1.add_rows(rows)
# print('\nLatextable Output:')
print(latextable.draw_latex(table_1, position='H', caption="Load Buffering benchmarks run with GenMC, WMC, and GenMC-XMM", label="table:lb-benchmarks"))


\begin{table}[H]
	\begin{center}
		\begin{tabular}{|l|c|c|c|c|c|c|}
			\hline
			Test Name & GenMC Execs & WMC Execs & WMC Dupls & XMM Execs & XMM Dupls & LB Races \\
			\hline
			LBn+ctrl(10) & 10 & 11 & 0 & 11 & 1 & 1 \\
			\hline
			LBn+ctrl(12) & 12 & 13 & 0 & 13 & 1 & 1 \\
			\hline
			LBn+ctrl(14) & 14 & 15 & 0 & 15 & 1 & 1 \\
			\hline
			LBn+data(10) & 1023 & 1024 & 0 & 1024 & 1 & 10 \\
			\hline
			LBn+data(12) & 4095 & 4096 & 0 & 4096 & 1 & 12 \\
			\hline
			LBn+data(14) & 16383 & 16384 & 0 & 16384 & 1 & 14 \\
			\hline
			LBn(10) & 1023 & 1024 & 9 & 1024 & 10 & 10 \\
			\hline
			LBn(12) & 4095 & 4096 & 11 & 4096 & 12 & 12 \\
			\hline
			LBn(14) & 16383 & 16384 & 13 & 16384 & 14 & 14 \\
			\hline
			LBn-pairs(10) & 243 & 1024 & 2101 & 1024 & 2184 & 2560 \\
			\hline
			LBn-pairs(12) & 729 & 4096 & 11529 & 4096 & 10379 & 12288 \\
			\hline
			LBn-pairs(14) & 2187 & 16384 & 61741 & 16384 & 48250 & 57344 \\
			\hline
		\end{tabular}
	\end{center}
	\caption{Load Buffering benc

In [5]:
print(table_1.draw())


+-----------+-----------+----------+----------+----------+----------+----------+
| Test Name |   GenMC   |   WMC    |   WMC    |   XMM    |   XMM    | LB Races |
|           |   Execs   |  Execs   |  Dupls   |  Execs   |  Dupls   |          |
+===========+===========+==========+==========+==========+==========+==========+
| LBn+ctrl( |    10     |    11    |    0     |    11    |    1     |    1     |
| 10)       |           |          |          |          |          |          |
+-----------+-----------+----------+----------+----------+----------+----------+
| LBn+ctrl( |    12     |    13    |    0     |    13    |    1     |    1     |
| 12)       |           |          |          |          |          |          |
+-----------+-----------+----------+----------+----------+----------+----------+
| LBn+ctrl( |    14     |    15    |    0     |    15    |    1     |    1     |
| 14)       |           |          |          |          |          |          |
+-----------+-----------+---